<a href="https://colab.research.google.com/github/pokjay/heb-squad/blob/main/finetune_mBERT_squad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Aug 27 12:50:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/pokjay/heb-squad

Cloning into 'heb-squad'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 64 (delta 19), reused 36 (delta 5), pack-reused 0
Unpacking objects: 100% (64/64), done.


In [ ]:
!gzip -d /content/heb-squad/data/final/heb-train-v2.0.csv.gz
!gzip -d /content/heb-squad/data/final/heb-dev-v2.0.csv.gz

In [ ]:
pip install transformers

     |████████████████████████████████| 2.6 MB 7.7 MB/s 
     |████████████████████████████████| 895 kB 58.5 MB/s 
     |████████████████████████████████| 636 kB 76.8 MB/s 
     |████████████████████████████████| 3.3 MB 71.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import collections
import pandas as pd
from tqdm.notebook import trange, tqdm

import torch
from torch.utils.data import DataLoader

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
device

device(type='cuda')

In [ ]:
BATCH_SIZE = 8

In [ ]:
df = pd.read_csv('/content/heb-squad/data/final/heb-train-v2.0.csv')
val_df = pd.read_csv('/content/heb-squad/data/final/heb-dev-v2.0.csv')

In [ ]:
df = df[(df.answer_start_heb > 0) & (df.answer_end_heb > 0)]
val_df = val_df[(val_df.answer_start_heb > 0) & (val_df.answer_end_heb > 0)]

Remove answer for impossible questions (The answer is actually the plausible answer)

In [ ]:
df['plausible_answer'] = df['answer']
df['plausible_answer_start_heb'] = df['answer_start_heb']
df['plausible_answer_end_heb'] = df['answer_end_heb']

val_df['plausible_answer'] = val_df['answer']
val_df['plausible_answer_start_heb'] = val_df['answer_start_heb']
val_df['plausible_answer_end_heb'] = val_df['answer_end_heb']

In [ ]:
df.loc[df.is_impossible == 1, 'answer'] = ''
df.loc[df.is_impossible == 1, 'answer_start_heb'] = 0
df.loc[df.is_impossible == 1, 'answer_end_heb'] = 0

val_df.loc[val_df.is_impossible == 1, 'answer'] = ''
val_df.loc[val_df.is_impossible == 1, 'answer_start_heb'] = 0
val_df.loc[val_df.is_impossible == 1, 'answer_end_heb'] = 0

In [ ]:
df.sample(1)

,id,context,question,answer,answer_start,answer_end,is_impossible,article,context_marked,answer_start_heb,answer_end_heb,plausible_answer,plausible_answer_start_heb,plausible_answer_end_heb
75183,5727b7c3ff5b5019007d9352,"סכר שלושת הנקיקים, הסכר הגדול ביותר שנבנה אי פ...",מה מטרתו של סכר שלושת הערוצים?,"לשלוט בהצפות באגן סצ'ואן, במחוז יונאן השכן ובמ...",117,202,0,Sichuan,"סכר שלושת הנקיקים, הסכר הגדול ביותר שנבנה אי פ...",88,141,"לשלוט בהצפות באגן סצ'ואן, במחוז יונאן השכן ובמ...",88,141


## Transform the texts to encodings

In [ ]:
train_ids = df.id.to_list()
train_is_impossible = df.is_impossible.to_list()
train_contexts = df.context.to_list()
train_questions = df.question.to_list()

val_ids = val_df.id.to_list()
val_is_impossible = val_df.is_impossible.to_list()
val_contexts = val_df.context.to_list()
val_questions = val_df.question.to_list()

In [ ]:
train_answers = df.apply(lambda x: {'answer_start' : x.answer_start_heb, 'answer_end': x.answer_end_heb}, axis=1).to_list()
val_answers = val_df.apply(lambda x: {'answer_start' : x.answer_start_heb, 'answer_end': x.answer_end_heb}, axis=1).to_list()

In [ ]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        if answers[i]['answer_end'] > 0:
          end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
        else:
          end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

In [ ]:
train_encodings.update({'id': train_ids})
val_encodings.update({'id': val_ids})

train_encodings.update({'is_impossible': train_is_impossible})
val_encodings.update({'is_impossible': val_is_impossible})

In [ ]:
from sys import getsizeof
getsizeof(train_answers)

910440

## Create PyTorch dataset

In [ ]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        # Get all encoded vals as tensors
        vals = {key: torch.tensor(val[idx]) for key, val in self.encodings.items() if key not in ('id', 'is_impossible')}
        # Add the id which is a string, used to map predictions to ids later
        vals.update({key: val[idx] for key, val in self.encodings.items() if key in ('id', 'is_impossible')})
        return vals

    def __len__(self):
        return len(self.encodings.input_ids)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

## Import Hebrew BERT for Q&A fine tuning

In [ ]:
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained("bert-base-multilingual-cased")

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-bas

In [ ]:
def load_checkpoint(path, model, optimizer):

  # Load checkpoint from Google Drive
  checkpoint = torch.load(path)

  # Load checkpoint to model and optimizer
  model.load_state_dict(checkpoint['state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer'])
  checkpoint_epoch = checkpoint['epoch']

  return checkpoint_epoch

## Code to evaluate model

In [ ]:
def calc_metrics(model_inf, dataloader):
  """
  Given a model and a dataset calculate the following metrics:
  - Exact Match
  - F1 Score
  - Loss
  @returns (F1, EM, Loss, EM Scores, F1 Scores)
  """

  model.to(device)
  model.eval()

  f1_scores = {0: {}, 1: {}}
  exact_scores = {0: {}, 1: {}}
  total_cnts = {0: 0, 1: 0}

  loss = 0
  total_cnt = 0

  for counter, batch in tqdm(enumerate(dataloader), total=len(dataloader)):
    with torch.no_grad():

      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      start_positions = batch['start_positions'].to(device)
      end_positions = batch['end_positions'].to(device)
      ids = batch['id']
      is_impossibles = batch['is_impossible']
      outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
      loss += outputs[0]

      pred_start_positions = torch.argmax(outputs['start_logits'], axis=1)
      pred_end_positions = torch.argmax(outputs['end_logits'], axis=1)

      # Calc scores
      for i in range(len(input_ids)):

        is_impossible = int(is_impossibles[i])

        total_cnts[is_impossible] += 1
        
        total_cnt += 1

        # Check if we have an exact match
        if start_positions[i] == pred_start_positions[i] and end_positions[i] == pred_end_positions[i]:
          # We have an exact match, mark it
          exact_scores[is_impossible][ids[i]] = 1
        else:
          # No match, mark with 0 or keep previous marking
          exact_scores[is_impossible][ids[i]] = max(0, exact_scores[is_impossible].get(ids[i], 0))

        # Get the predicted answer token sequence
        pred_tokens = input_ids[i][pred_start_positions[i]:pred_end_positions[i] + 1].tolist()
        answer_tokens = input_ids[i][start_positions[i]:end_positions[i] + 1].tolist()

        # Check how many predicted tokens correspond to gold answer tokens
        common = collections.Counter(answer_tokens) & collections.Counter(pred_tokens)
        num_same = sum(common.values())

        # If none same, then F1=0 , else use the F1 formula
        if num_same == 0:
          score = 0
        else:
          precision = 1.0 * num_same / len(pred_tokens)
          recall = 1.0 * num_same / len(answer_tokens)
          score = (2 * precision * recall) / (precision + recall)
        
        # We take the max F1 score of the gold answers
        f1_scores[is_impossible][ids[i]] = max(score, f1_scores[is_impossible].get(ids[i], 0))

  f1_joined = {**f1_scores[0], **f1_scores[1]}
  em_joined = {**exact_scores[0], **exact_scores[1]}

  f1 = sum(f1_joined.values()) / len(f1_joined)
  em = sum(em_joined.values()) / len(em_joined)

  f1_split = {0: round(sum(f1_scores[0].values()) / len(f1_scores[0]), 4),
              1: round(sum(f1_scores[1].values()) / len(f1_scores[1]), 4),
              'all': round(f1, 4)}

  em_split = {0: round(sum(exact_scores[0].values()) / len(exact_scores[0]), 4),
              1: round(sum(exact_scores[1].values()) / len(exact_scores[1]), 4),
              'all': round(em, 4)}

  epoch_avg_loss = loss / len(dataloader)

  metrics = {'f1': f1_split,
             'em': em_split,
             'loss': epoch_avg_loss}

  return metrics

## Train model

In [ ]:
from transformers import AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)

optim = AdamW(model.parameters(), lr=5e-5)

train_loss = []
val_loss = []

start_epoch = 0

# checkpoint_path = '/content/drive/MyDrive/nlp/checkpoints/checkpoint_0_0.501_0.462_1.841_2.159'
# start_epoch = load_checkpoint(checkpoint_path, model, optim) + 1

print(f'Training model from epoch={start_epoch}')

for epoch in range(start_epoch, 5):

    model.train()

    epoch_loss = 0.0    # Cummulative loss for epoch
    running_loss = 0.0  # Loss per averaging size

    for counter, batch in enumerate(train_loader):
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]
        loss.backward()
        optim.step()
        
        running_loss += loss.item()
        epoch_loss += loss.item()

        if counter % 250 == 0 and counter > 0:
            print('Epoch %d, %d/%d, Current Loss = %.4f' % (epoch, counter, len(train_loader), running_loss / 250))
            running_loss = 0.0

    avg_train_loss = epoch_loss / len(train_loader)

    print('Epoch %d, Train Loss = %.4f' % (epoch, avg_train_loss))

    metrics = calc_metrics(model, val_loader)

    f1 = metrics['f1']['all']
    em = metrics['em']['all']
    avg_val_loss = metrics['loss']

    print('Epoch %d, Val Loss = %.4f, F1 = %.4f, EM=%.4f' % (epoch, avg_val_loss, f1, em))
    print(metrics)

    train_loss.append(avg_train_loss)
    val_loss.append(avg_val_loss)

    state = {
        'epoch': epoch,
        'state_dict': model.state_dict(),
        'optimizer': optim.state_dict(),
        'f1': f1,
        'em': em,
        'avg_val_loss': avg_val_loss,
        'avg_train_loss': avg_train_loss
    }
    savepath=f'/content/drive/MyDrive/nlp/checkpoints/finetune_mBERT_new_dataset/checkpoint_{epoch}_{f1:.3f}_{em:.3f}_{avg_train_loss:.3f}_{avg_val_loss:.3f}'
    torch.save(state, savepath)

Training model from epoch=0
Epoch 0, 250/14225, Current Loss = 3.0767
Epoch 0, 500/14225, Current Loss = 2.5763
Epoch 0, 750/14225, Current Loss = 2.3189
Epoch 0, 1000/14225, Current Loss = 2.1881
Epoch 0, 1250/14225, Current Loss = 2.0936
Epoch 0, 1500/14225, Current Loss = 2.0132
Epoch 0, 1750/14225, Current Loss = 2.0819
Epoch 0, 2000/14225, Current Loss = 2.0402
Epoch 0, 2250/14225, Current Loss = 1.9844
Epoch 0, 2500/14225, Current Loss = 1.9345
Epoch 0, 2750/14225, Current Loss = 1.9897
Epoch 0, 3000/14225, Current Loss = 1.9208
Epoch 0, 3250/14225, Current Loss = 1.9179
Epoch 0, 3500/14225, Current Loss = 1.8824
Epoch 0, 3750/14225, Current Loss = 1.8703
Epoch 0, 4000/14225, Current Loss = 1.8155
Epoch 0, 4250/14225, Current Loss = 1.8113
Epoch 0, 4500/14225, Current Loss = 1.8682
Epoch 0, 4750/14225, Current Loss = 1.8790
Epoch 0, 5000/14225, Current Loss = 1.8657
Epoch 0, 5250/14225, Current Loss = 1.7727
Epoch 0, 5500/14225, Current Loss = 1.7508
Epoch 0, 5750/14225, Current 

  0%|          | 0/2826 [00:00<?, ?it/s]

Epoch 0, Val Loss = 1.8941, F1 = 0.5581, EM=0.5109
{'f1': {0: 0.4388, 1: 0.6867, 'all': 0.5581}, 'em': {0: 0.3517, 1: 0.6826, 'all': 0.5109}, 'loss': tensor(1.8941, device='cuda:0')}
Epoch 1, 250/14225, Current Loss = 1.4429
Epoch 1, 500/14225, Current Loss = 1.4795
Epoch 1, 750/14225, Current Loss = 1.5045
Epoch 1, 1000/14225, Current Loss = 1.5077
Epoch 1, 1250/14225, Current Loss = 1.4796
Epoch 1, 1500/14225, Current Loss = 1.5365
Epoch 1, 1750/14225, Current Loss = 1.4611
Epoch 1, 2000/14225, Current Loss = 1.5197
Epoch 1, 2250/14225, Current Loss = 1.4022
Epoch 1, 2500/14225, Current Loss = 1.4432
Epoch 1, 2750/14225, Current Loss = 1.4258
Epoch 1, 3000/14225, Current Loss = 1.4400
Epoch 1, 3250/14225, Current Loss = 1.5563
Epoch 1, 3500/14225, Current Loss = 1.5272
Epoch 1, 3750/14225, Current Loss = 1.5437
Epoch 1, 4000/14225, Current Loss = 1.4228
Epoch 1, 4250/14225, Current Loss = 1.4655
Epoch 1, 4500/14225, Current Loss = 1.5169
Epoch 1, 4750/14225, Current Loss = 1.4573
Epo

  0%|          | 0/2826 [00:00<?, ?it/s]

Epoch 1, Val Loss = 1.8369, F1 = 0.5798, EM=0.5343
{'f1': {0: 0.4312, 1: 0.7399, 'all': 0.5798}, 'em': {0: 0.3464, 1: 0.7368, 'all': 0.5343}, 'loss': tensor(1.8369, device='cuda:0')}
Epoch 2, 250/14225, Current Loss = 1.3146
Epoch 2, 500/14225, Current Loss = 1.2637
Epoch 2, 750/14225, Current Loss = 1.2054
Epoch 2, 1000/14225, Current Loss = 1.2222
Epoch 2, 1250/14225, Current Loss = 1.1884
Epoch 2, 1500/14225, Current Loss = 1.3212
Epoch 2, 1750/14225, Current Loss = 1.2168
Epoch 2, 2000/14225, Current Loss = 1.2184
Epoch 2, 2250/14225, Current Loss = 1.3184
Epoch 2, 2500/14225, Current Loss = 1.2827
Epoch 2, 2750/14225, Current Loss = 1.2091
Epoch 2, 3000/14225, Current Loss = 1.3538
Epoch 2, 3250/14225, Current Loss = 1.2634
Epoch 2, 3500/14225, Current Loss = 1.2607
Epoch 2, 3750/14225, Current Loss = 1.3278
Epoch 2, 4000/14225, Current Loss = 1.2582
Epoch 2, 4250/14225, Current Loss = 1.3174
Epoch 2, 4500/14225, Current Loss = 1.2949
Epoch 2, 4750/14225, Current Loss = 1.2923
Epo

...

Epoch 2, Val Loss = 1.8558, F1 = 0.5997, EM=0.5427
{'f1': {0: 0.504, 1: 0.7029, 'all': 0.5997}, 'em': {0: 0.3992, 1: 0.6974, 'all': 0.5427}, 'loss': tensor(1.8558, device='cuda:0')}
Epoch 3, 250/14225, Current Loss = 1.0843
Epoch 3, 500/14225, Current Loss = 1.0696
Epoch 3, 750/14225, Current Loss = 1.1025
Epoch 3, 1000/14225, Current Loss = 1.0850
Epoch 3, 1250/14225, Current Loss = 1.0797
Epoch 3, 1500/14225, Current Loss = 1.0822
Epoch 3, 1750/14225, Current Loss = 1.1339
Epoch 3, 2000/14225, Current Loss = 1.1424
Epoch 3, 2250/14225, Current Loss = 1.1593
Epoch 3, 2500/14225, Current Loss = 1.0756
Epoch 3, 2750/14225, Current Loss = 1.1110
Epoch 3, 3000/14225, Current Loss = 1.1428
Epoch 3, 3250/14225, Current Loss = 1.0961
Epoch 3, 3500/14225, Current Loss = 1.1319
Epoch 3, 3750/14225, Current Loss = 1.1383
Epoch 3, 4000/14225, Current Loss = 1.1353
Epoch 3, 4250/14225, Current Loss = 1.1427
Epoch 3, 4500/14225, Current Loss = 1.1488
Epoch 3, 4750/14225, Current Loss = 1.1622
Epoc

  0%|          | 0/2826 [00:00<?, ?it/s]

Epoch 3, Val Loss = 1.7614, F1 = 0.6000, EM=0.5412
{'f1': {0: 0.5588, 1: 0.6444, 'all': 0.6}, 'em': {0: 0.4506, 1: 0.639, 'all': 0.5412}, 'loss': tensor(1.7614, device='cuda:0')}
Epoch 4, 250/14225, Current Loss = 0.9666
Epoch 4, 500/14225, Current Loss = 0.9196
Epoch 4, 750/14225, Current Loss = 0.9748
Epoch 4, 1000/14225, Current Loss = 0.9587
Epoch 4, 1250/14225, Current Loss = 0.9894
Epoch 4, 1500/14225, Current Loss = 0.9751
Epoch 4, 1750/14225, Current Loss = 1.0456
Epoch 4, 2000/14225, Current Loss = 0.9800
Epoch 4, 2250/14225, Current Loss = 1.0226
Epoch 4, 2500/14225, Current Loss = 1.0203
Epoch 4, 2750/14225, Current Loss = 0.9732
Epoch 4, 3000/14225, Current Loss = 1.0111
Epoch 4, 3250/14225, Current Loss = 0.9991
Epoch 4, 3500/14225, Current Loss = 0.9490
Epoch 4, 3750/14225, Current Loss = 1.0542
Epoch 4, 4000/14225, Current Loss = 0.9392
Epoch 4, 4250/14225, Current Loss = 1.0484
Epoch 4, 4500/14225, Current Loss = 1.0284
Epoch 4, 4750/14225, Current Loss = 0.9832
Epoch 4

  0%|          | 0/2826 [00:00<?, ?it/s]

Epoch 4, Val Loss = 1.8178, F1 = 0.5981, EM=0.5337
{'f1': {0: 0.5862, 1: 0.611, 'all': 0.5981}, 'em': {0: 0.4688, 1: 0.6037, 'all': 0.5337}, 'loss': tensor(1.8178, device='cuda:0')}
